In [1]:
import pandas as pd
import numpy as np
import re
from langdetect import detect
from sklearn.model_selection import train_test_split
import cld3
import math
from dataset import *
from main import *

In [449]:
train_subset = train.iloc[:1000]

In [450]:
train_subset.to_csv('data/train_subset.csv', index=False)

In [5]:
train = pd.read_csv("data/csv/train.csv")
train.head()

,Artist,Song,Genre,Language,Lyrics,reliable_lang
0,belle sebastian,legal man,Pop,en,"L-O-V-E love, it's coming back, it's coming ba...",en
1,pete townshend,a friend is a friend,Rock,en,When eyes meet in silence\nA pact can be made\...,en
2,blossoms,love talk,Rock,en,my eyes align with you\nyou're on my side\nand...,en
3,bukka white,parchman farm blues,Folk,en,Judge gimme me life this morn'in\nDown on Parc...,en
4,pat benatar,i won't,Rock,en,I was there when you cried like a baby\nWhen y...,en


In [315]:
data = pd.read_csv("data/train.csv")
data = data[data['Language']=='en']
data = data[data['Lyrics'].apply(lambda x: '\n' in x)]
data = data.drop_duplicates(subset=['Artist', 'Song', 'Genre'], ignore_index=True)
data = data[data['Lyrics'].apply(lambda x: '---------' not in x)]
data['Lyrics'] = data['Lyrics'].apply(lambda x: re.sub('\[.*\]', ' ', x))

In [316]:
def check_en(line):
    result = cld3.get_language(line)
    lang = result[0]
    reliable = result[2]
    if reliable and lang == 'en':
        return 'en'
    else:
        return 'other'

In [317]:
data['reliable_lang'] = data['Lyrics'].apply(check_en)
data = data[data['reliable_lang']=='en']

In [321]:
indices = np.arange(len(data))
train_index, test_index = train_test_split(indices, test_size = 0.1)
train_index, dev_index = train_test_split(train_index, test_size=0.1)

train_data = data.iloc[train_index].reset_index(drop=True)
dev_data = data.iloc[dev_index].reset_index(drop=True)
test_data = data.iloc[test_index].reset_index(drop=True)

In [322]:
train_data.to_csv('data/csv/train.csv', index=False)
dev_data.to_csv('data/csv/dev.csv', index=False)
test_data.to_csv('data/csv/test.csv', index=False)

In [294]:
data['Genre'].value_counts()

Rock          96376
Pop           70438
Metal         15910
Jazz          11701
Folk           6922
Indie          5592
R&B            1572
Electronic      447
Name: Genre, dtype: int64

In [288]:
df = pd.read_csv('data/csv/test.csv')

In [290]:
df = df[df['Genre']=='Pop']

In [280]:
dev_data['Genre'].value_counts()

Rock          8679
Pop           6313
Metal         1490
Jazz          1035
Folk           603
Indie          497
R&B            171
Electronic      36
Name: Genre, dtype: int64

In [184]:
jazz = data[data['Genre']=='Jazz'].reset_index(drop=True)
indices = np.arange(len(jazz))
train_index, test_index = train_test_split(indices, test_size = 0.1)
train_index, dev_index = train_test_split(train_index, test_size=0.1)

In [242]:
path = 'data/train/jazz_train.txt'
with open(path,'w', encoding='utf-8', errors='ignore') as f:
    for i in train_index:
#         f.write("ARTIST NAME: "+jazz['Artist'].iloc[i]+'\n')
#         f.write("GENRE: "+jazz['Genre'].iloc[i]+'\n')
        f.write(jazz['Lyrics'].iloc[i])
        f.write('\n')

In [116]:
for g in list(data['Genre'].value_counts().index):
    df = data[data['Genre']==g]
    artist = np.array(df['Artist']).reshape(-1,1)
    genre = np.array(df['Genre']).reshape(-1,1)
    lyrics = np.array(df['Lyrics']).reshape(-1,1)

    out = np.hstack((artist, genre, lyrics))
    file_path = 'data/'+g+'_train.npy'
    np.save(file_path, out)

In [105]:
artist = np.array(rnb['Artist']).reshape(-1,1)
genre = np.array(rnb['Genre']).reshape(-1,1)
lyrics = np.array(rnb['Lyrics']).reshape(-1,1)

rnb_out = np.hstack((artist, genre, lyrics))
np.save('data/rnb_train.npy', rnb_out)

In [93]:
detect(rnb['Lyrics'].iloc[0])

'en'

In [90]:
rnb['detect_lang'] = rnb['Lyrics'].apply(lambda x: detect(x[:50]))

/Users/haowei/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [80]:
data['Genre'].value_counts()

Rock          103721
Pop            83620
Metal          17207
Jazz           11760
Folk            7023
Indie           6033
R&B             1575
Electronic       462
Name: Genre, dtype: int64

In [442]:
rnb = create_ngram_model(NgramModel, 'data/csv/train.csv', 4,0.0000001,genre='R&B')

In [437]:
rnb6 = create_ngram_model(NgramModel, 'data/csv/train.csv', 6,0.0000001,genre='R&B')

In [443]:
rnb.gen_next_sent('I like')

'I will in my loaded came'

In [396]:
test_df = pd.read_csv('data/csv/test.csv')
train_df = pd.read_csv('data/csv/train.csv')

In [351]:
idx = random.randint(0,len(test_df)-1)
lyric = test_df.iloc[idx]['Lyrics']
# tot_pp += model.perplexity(lyric)

In [2]:
spec_dict = {"dropout": 0.5,
                 "num_lstm_layers": 2,
                 "bilstm_flag": True,
                 "word_bilstm_flag": False,
                 "use_artist": True,
                 "char_hidden_dim": 128,
                 "char_emb_dim": 100,
                 "char_model_type": "LSTM",
                 "word_emb_dim": 256,
                 "pre_train_word_embedding": None,
                 "feature_emb_dim": 128,
                 "final_hidden_dim": 512,
                 "learning rate": 0.001,
                 "iterations": 5000,
                 "print_every": 200,
                 "plot_every": 100
                 }
ds = Dataset('./data/csv/train.csv', subset=['R&B'])
vocab_size = ds.tokenize_corpus(word_tokenize)
print("Successfully built dataset...")
print("index of new line character: ", ds.word2idx['\n'])
print("index of comma: ", ds.word2idx[','])
print("index of <UNK>: ", ds.word2idx['<UNK>'])
ds.save_dict()

Successfully built dataset...
index of new line character:  8069
index of comma:  8769
index of <UNK>:  10688


In [10]:
with open('saved_data/artist_set.p', "wb") as fp:
    pickle.dump(ds.artist_set, fp)
with open('saved_data/genre_set.p', 'wb') as fp:
    pickle.dump(ds.genre_set, fp)
with open('saved_data/word2idx.p', 'wb') as fp:
    pickle.dump(ds.word2idx, fp)
with open('saved_data/idx2word.p', 'wb') as fp:
    pickle.dump(ds.idx2word, fp)
with open('saved_data/phones2index.p', 'wb') as fp:
    pickle.dump(ds.phones2index, fp)
with open('saved_data/index2phones.p', 'wb') as fp:
    pickle.dump(ds.index2phones, fp)

In [4]:
model = WordSequence(spec_dict, ds)
optimizer = optim.Adam(model.parameters(), lr=spec_dict['learning rate'])
model, loss_list = train(model, optimizer, ds, spec_dict, num_lines=3)
path = 'saved_parameters1'
torch.save({'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, path)

loss in current iteration:  3.297504425048828
Input string:  Anymore . 
 When that indigo 
 I can't deny that I'm blue
Target string:  . 
 When that indigo 
 I can't deny that I'm blue .
prediced tensor:  tensor([[7405]])
decoded string:  me 
loss in current iteration:  10.1879301071167
Input string:  You are my shelter , from the pouring rain 
 You were my comfort , even before the pain 
 I can hear the sound of five drummers in the
Target string:  are my shelter , from the pouring rain 
 You were my comfort , even before the pain 
 I can hear the sound of five drummers in the wind
prediced tensor:  tensor([[7405]])
decoded string:  me 
loss in current iteration:  3.176255464553833
Input string:  Games 
 Not goin' out like that let me explain 
 Tryin' to save myself don't pressure
Target string:  
 Not goin' out like that let me explain 
 Tryin' to save myself don't pressure me
prediced tensor:  tensor([[8769]])
decoded string:  , 
loss in current iteration:  6.339225769042969
Input s

In [25]:
spec_dict = {"dropout": 0.7,
                 "num_lstm_layers": 2,
                 "bilstm_flag": True,
                 "word_bilstm_flag": False,
                 "use_artist": True,
                 "char_hidden_dim": 128,
                 "char_emb_dim": 100,
                 "char_model_type": "LSTM",
                 "word_emb_dim": 256,
                 "pre_train_word_embedding": None,
                 "feature_emb_dim": 128,
                 "final_hidden_dim": 512,
                 "learning rate": 0.001,
                 "iterations": 1000,
                 "print_every": 100,
                 "plot_every": 50
                 }
model, loss_list = train(model, ds, spec_dict, num_lines=2)
torch.save(model.state_dict(), 'model4.pt')

loss in current iteration:  4.775262355804443
Input string:  Superman can fly high way up in the sky 
 'Cause we believe he can 
 So what we choose to believe can always work out fine 
 It's all in the mind 
 So think of a place and imagine a time 
 And let's go be lovers 
 Believing me , Ooo Wee baby 
 Is believing right - oh yeah , and 
 I wanna love , wanna have , wanna hold you girl 
 So make me a believer 
 Wanna love you girl 
 Ooo so near 
 And forever keep you here 
 Forget all about whatever we fear 
 And let's just be lovers 
 Loving and loving , oh love is the way 
 To make me stay 
 I know the way to persuade me over to your side 
 And I am sure you can 
 So if you get me to believe 
 It all will work out fine 
 'Cause you'll blow my mind 
 So let's pick a place , a day , and a time 
 And then we'll be lovers 
 Loving and loving , oh love is the way 
 To make me
Target string:  can fly high way up in the sky 
 'Cause we believe he can 
 So what we choose to believe can alwa

loss in current iteration:  5.075776100158691
Input string:  the world is gon' turn . the world is on & on 
 the world is gon' turn . turn 
 and the world is on & on 
 all yeah oh yeah . yeah 
 on . on . on . on . on . on and on . on . on and on . on . on . on and on 
 on . on . on and on . the world is gon' turn . the world is on and on
Target string:  world is gon' turn . the world is on & on 
 the world is gon' turn . turn 
 and the world is on & on 
 all yeah oh yeah . yeah 
 on . on . on . on . on . on and on . on . on and on . on . on . on and on 
 on . on . on and on . the world is gon' turn . the world is on and on .
prediced tensor:  tensor([[ 1277, 10503,  9895,  5718,  9754,  9754,  2520,  9754,  9895,  9895,
          9754,  9895,  6092,  8851, 10503,  9895,  5718,  9754,  9754,  9754,
          6092,  6092,  2520,  9754,  9895,  9895,  9754,  9895,  6092,  9895,
          9754,  9754,  9754,  9754,  9754,  6092,  9895,  9754,  9754,  9754,
          9754,  9754,  9754,  97

loss in current iteration:  4.348729610443115
Input string:  Stop searchin' , no need lookin' around 
 Got what you need , stop foolin' around 
 Everything you need , I already found 
 And it's so lovely I can't wait for you to see 
 How this love has changed my life , my life 
 Make me wanna change wrong from right , from right 
 And I love it I just wanna share it 
 I promise that you won't regret it 
 You been searchin' for a long , long time 
 You looked there but not here and I will bet if 
 You have this love in you 
 You'll be tellin' everybody , like I'm tellin' you 
 Real love , real love , I got it and I gotta shout it 
 And let everybody know about it 
 Real love , real love , I got it an' I'm so glad I found it 
 And I won't let it go 
 Real love , real love , I got it and I gotta shout it 
 And let everybody know about it 
 Real love , real love , I got it an' I'm so glad I found it 
 And I won't let it go 
 I used to be you , lookin' and searchin' 
 Just tryin' to find so

char not found in printable characters:  déjà
char not found in printable characters:  déjà
char not found in printable characters:  déjà
char not found in printable characters:  déjà
char not found in printable characters:  déjà
char not found in printable characters:  déjà
loss in current iteration:  5.08379602432251
Input string:  Lately I've been thinkin' 
 Thinking 'bout you 
 And all the things 
 I've seen you go through 
 Your mother the kids and 
 The problems at home 
 Sorry I wish I could fix what's wrong 
 I hurt when you hurt and 
 I cry when you cry 
 Even the deep ones 
 Sometimes wonder why am I going through 
 I'm waiting but still no use 
 
 He's still in control 
 He's sovereign and He knows 
 Just how it feels to be afraid 
 Have folk you love walk away 
 Be still and know He's still in control 
 
 Lately depression 
 Your job and your life 
 Weighs on your mind 
 All day and all night 
 You know every scripture 
 And what prayer to pray 
 Only a fool would think 
 I

loss in current iteration:  4.197919845581055
Input string:  I don't have to say you're beautiful 
 You've got plenty other guys to say those words for you 
 And I don't have to say you run the show 
 All it takes is a word , I lose control for you 
 
 I know you're not my girlfriend , but 
 I swear that I love you 
 Baby I do 
 I know you're not my girlfriend , but 
 I swear that I love you 
 Baby I do 
 I don't have to say you stole my heart 
 Cause anything I have I freely give to you 
 And I knew right from the very start 
 That the only place I belong is next to you 
 
 Even through all 
 The ups and downs 
 You can't stop 
 The way I feel now 
 Even through the storm 
 Joy and pain 
 I won't change 
 Still feel the same 
 Anytime you call 
 I don't care 
 No matter the reason 
 I'll be there 
 Constantly on my mind 
 And that's why I'm hoping 
 You will be my 
 Valentine ( valentine ) 
 
 And I know you love me too 
 I swear that I love you 
 Yes I
Target string:  don't have to s

In [12]:
saved_model = model

In [129]:
inp, target, artist, genre, next_line = ds.random_lyric_chunks(path = "data/csv/train.csv", subset=["R&B"], num_lines=4, if_train=False)

In [6]:
inp, target, artist, genre, next_line = ds.random_lyric_chunks(path = "data/csv/train.csv", subset=["R&B"], num_lines=2, if_train=False)
# inp = "Are you ready ? "
# target = inp
input_list = gen_input(ds, inp, target, artist, genre, if_train=False)
res = batchify_sequence_labeling(input_list, False)
(word_seq_tensor, feature_seq_tensors, word_seq_lengths, word_seq_recover, char_seq_tensor, char_seq_lengths,
     char_seq_recover, ph_inputs, ph_seq_lengths, ph_seq_recover, target_seq_tensor, mask) = res
batch_size = word_seq_tensor.size(0)
seq_len = word_seq_tensor.size(1)
genre_input = feature_seq_tensors[1]
artist_input = feature_seq_tensors[0]
outs = model(word_seq_tensor, genre_input, artist_input, word_seq_lengths, char_seq_tensor, char_seq_lengths, char_seq_recover,
        ph_inputs, ph_seq_lengths, ph_seq_recover)
outs = outs.view(batch_size * seq_len, -1)
score = F.log_softmax(outs, 1)
_, pred = torch.max(score, 1)
pred = pred.view(batch_size, seq_len)
print("input: ", inp)
print("next_line: ", next_line)
print("predicted lyrics: ")
print(decode_lyrics(ds, pred[0]))

NameError: name 'saved_model' is not defined

In [108]:
score.size()

torch.Size([4, 10689])

In [115]:
score = F.log_softmax(o1,1)
_, pred = torch.max(score, 1)

In [116]:
pred

tensor([10578])

In [100]:
loss_function = nn.NLLLoss(ignore_index=0)

In [101]:
pred

tensor([10578, 10578, 10578, 10578])

In [89]:
target_seq_tensor.view(4)[-1].view(1).size()

torch.Size([1])

In [114]:
target_seq_tensor

tensor([[5748, 6473, 7967,  788]])

In [113]:
loss_function(score, target_seq_tensor.view(4)[-1].view(1))

tensor(4.7541, grad_fn=<NllLossBackward>)

In [63]:
target_seq_tensor.view(4)[-1]

tensor(788)

In [60]:
decode_lyrics(ds, target_seq_tensor.view(4))

'are you ready ? '

In [53]:
ds.all_characters.index('\n')

96

In [46]:
_, idx = torch.max(score, 1)

In [48]:
max(score[0])

tensor(-2.3579, grad_fn=<UnbindBackward>)

In [50]:
score[0][score[0] > -2.4]

tensor([-2.3579], grad_fn=<IndexBackward>)

In [47]:
idx

tensor([10578, 10578, 10578, 10578])

In [45]:
decode_lyrics(ds, idx)

'scented unbeatable scented motivating '

In [34]:
decode_lyrics(ds, pred[0])

'\n \n \n \n '

In [9]:
pred_lines = predict(model, ds)
print(pred_lines)

Start predicting next line..
prediction input:  Our love is like water/angels 
 Pinned down and abused 

prediction tensor:  tensor([[5064, 7653, 2464, 7563, 5617, 3331, 3863, 9343, 4211, 5617, 2971]])
oh
full word
input len to lstm generation:  12
prediction tensor:  tensor([[5064, 7653, 2464, 7563, 5617, 8930, 3863, 9343, 7563, 5617, 2971, 1330]])
yeah
full word
input len to lstm generation:  13
prediction tensor:  tensor([[5064, 7653, 2464, 7563, 5617, 8930, 3863, 9343, 7563, 5617, 2971, 1330,
         9474]])
me
full word
input len to lstm generation:  14
prediction tensor:  tensor([[ 5064,  7653,  2464,  7563,  5617,  8930,  3863,  9343,  7563,  5617,
          2971, 10420,  9474,  7643]])
)
full word
input len to lstm generation:  15
prediction tensor:  tensor([[ 5064,  7653,  2464,  7563,  5617,  8930,  3863,  9343,  7563,  5617,
          2971, 10420,  9474,  7643,  9616]])
.
full word
input len to lstm generation:  16
prediction tensor:  tensor([[ 5064,  7653,  2464,  7563,  5

In [19]:
(word_seq_tensor, feature_seq_tensors, word_seq_lengths, word_seq_recover, char_seq_tensor, char_seq_lengths, char_seq_recover, phone_seq_tensor, phone_seq_lengths, phone_seq_recover, target_seq_tensor, mask) = res

In [20]:
lr = 0.001
model = WordSequence(spec_dict, ds)
optimizer = optim.Adam(model.parameters(), lr=lr)

build char sequence feature extractor: LSTM ...
build char sequence feature extractor: LSTM ...


In [12]:
word_seq_tensor

tensor([[7676, 2313, 3358, 2017, 8924, 7537, 7537, 7676, 2313, 3358, 2017, 8924,
         7537]])

In [13]:
char_seq_tensor.size()

torch.Size([13, 6])

In [21]:
model(word_seq_tensor, feature_seq_tensors[0], feature_seq_tensors[1], word_seq_lengths, char_seq_tensor, char_seq_lengths, char_seq_recover, phone_seq_tensor, phone_seq_lengths, phone_seq_recover)


IndexError: index out of range in self

In [30]:
s = "Said I'm sorry baby ,  said forgive me baby\nNow you ain't never seen me on one"
tokens = re.split("[ ]+", s)

In [31]:
tokens

['Said',
 "I'm",
 'sorry',
 'baby',
 ',',
 'said',
 'forgive',
 'me',
 'baby\nNow',
 'you',
 "ain't",
 'never',
 'seen',
 'me',
 'on',
 'one']